In [1]:
import os
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import OllamaLLM
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import DirectoryLoader, PyPDFLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [2]:
# I've stored my LANGCHAIN_API_KEY definition in a separate file 
from langsmithkey import *

In [3]:
matt_path=Path(os.getcwd())
root_path=matt_path.parents[0]
db_path=root_path.joinpath(r'data\vectorstore\db_faiss')
word_path=root_path.joinpath(r'data\apra_standards\word')
print(matt_path)
print(root_path)
print(db_path)
print(word_path)

d:\repos\ChatBot-Adv-NLP\Matt
d:\repos\ChatBot-Adv-NLP
d:\repos\ChatBot-Adv-NLP\data\vectorstore\db_faiss
d:\repos\ChatBot-Adv-NLP\data\apra_standards\word


In [4]:
llm_model ="llama3.2"
embedding_model='sentence-transformers/all-MiniLM-L6-v2'
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

# From https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.faiss.FAISS.html#langchain_community.vectorstores.faiss.FAISS.load_local
#       allow_dangerous_deserialization (bool) 
#           whether to allow deserialization of the data which involves loading a pickle file. 
#           Pickle files can be modified by malicious actors to deliver a malicious payload 
#           that results in execution of arbitrary code on your machine.

# As I had built this database from scratch, I have no concerns over the use of pickle files...
vectorstore = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

c:\ProgramData\anaconda3\envs\chain\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
llm = OllamaLLM(model="llama3.2")

In [6]:
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is RWA?")

"Risk-weighted assets (RWA) refer to the amount of a bank's assets that are subject to specific capital requirements. RWA is calculated using standardized approaches or internal rating-based approaches and is used to determine an institution's capital adequacy. It includes securitisation exposures, operational risk, market risk, and counterparty credit risk."